# 0.
https://www.bilibili.com/video/BV18g4119737?p=37&vd_source=70200f7d09862fd682e5f89b22c89125

# 1. 梯度介绍
## 1.1 几种情况
1. local minimum ![](https://gitee.com/wyjyoga/my-pic-go/raw/master/img/20221111205923.png)
	- resnet通过“短路”的设置加深网络深度，从而达到很好的效果
	- 是因为加深之后，loss平面变得平滑（右侧），更加容易找到全局最优

2. Saddel point
![|325](https://gitee.com/wyjyoga/my-pic-go/raw/master/img/20221111210352.png)
这里x和y是两个面，鞍点是x的极小值但是是y的最大值。这种情况是很多的

## 1.2 Optimizer performance（影响优化的因素）
1. Initialization status
   ![](https://gitee.com/wyjyoga/my-pic-go/raw/master/img/20221111210747.png)
   - 提到了**kaiming**初始方法
2. learning rate
   - 同时影响<u>速度和精度</u>
   - 要衰减 decay
3. momentum

# 2. 激活函数&梯度
1. sigmoid
   - 缺陷：**梯度弥散**：当x比较大时，梯度很小，参数长时间得不到

In [6]:
import torch
from torch.nn import functional as F

In [3]:
# 返回一个1维张量，包含在区间start和end上均匀间隔的step个点。
# 输出张量的长度由steps决定。
# aka. 等差数列
a = torch.linspace(-100,100,10)
a

tensor([-100.0000,  -77.7778,  -55.5556,  -33.3333,  -11.1111,   11.1111,
          33.3333,   55.5556,   77.7778,  100.0000])

In [4]:
torch.sigmoid(a)

tensor([0.0000e+00, 1.6655e-34, 7.4564e-25, 3.3382e-15, 1.4945e-05, 9.9999e-01,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00])

2. Tanh：
    - 导数也和tanh直接有关
    - tanh = 2*sigmoid(2x)-1

In [5]:
a = torch.linspace(-1,1,10)
torch.tanh(a)

tensor([-0.7616, -0.6514, -0.5047, -0.3215, -0.1107,  0.1107,  0.3215,  0.5047,
         0.6514,  0.7616])

3. RELU
    - 最简单最普遍最优先
    - why work：x>=0时，**不会对梯度放大或者缩小**，梯度弥散和爆炸的情况得到很大缓解

In [8]:
print(a)
F.relu(a)

tensor([-1.0000, -0.7778, -0.5556, -0.3333, -0.1111,  0.1111,  0.3333,  0.5556,
         0.7778,  1.0000])


tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1111, 0.3333, 0.5556, 0.7778,
        1.0000])

# 3. Loss函数&梯度
## 3.1 MSE
1. MSE = L2_norm^2 = torch.norm((y-(xw+b)),2)^2
2. 注意：
    - code时要对求导的变量设置为`requires_grad = True`或者使用`w.requires_grad_()`

In [39]:
x = torch.ones(1)
w = torch.tensor([2.],requires_grad = True)
# w = torch.full([1].float(),2)
# w.requires_grad_()
mse = F.mse_loss(torch.ones(1),x*w)
x,w,mse

(tensor([1.]),
 tensor([2.], requires_grad=True),
 tensor(1., grad_fn=<MseLossBackward0>))

3. 手动算梯度be like: `autograd.grad()`
    - 或者`[w1,w2,w3...]`

In [37]:
torch.autograd.grad(mse,[w])

(tensor([2.]),)

4. 大多数情况下不需要手动算梯度，直接调用：
    - 此时不会返回grad值，而是默默记录在`w.grad`上
    - 由于梯度发生累积，这里不再是2而是4(or more)
    - 在实际中，会打印出梯度的`norm`来观察，毕竟梯度有时候可能dim很大

In [42]:
mse = F.mse_loss(torch.ones(1),x*w)
mse.backward()
print(w.grad)
print(w.grad.norm)

tensor([6.])
<bound method Tensor.norm of tensor([6.])>


## 3.2 Cross Entropy Loss
1. 单分类多分类用的都很多
2. 分类：
    - binary
    - multi-class
    - softmax【本节】
    - leave it to LR part
3. softmax = soft version of max：
    - 有一个（金字塔）放大作用：原来大score的现在（prob）更大
    - 设 $softmax(\alpha_j) = p_i$，则对$\alpha_j$求导（j是第j个节点）
        - 当$i=j,p_i(1-p_i)$，一定是个正值
        - 当$i!=j, -p_j*p_i$，一定是个负值

In [44]:
a = torch.randn(3,requires_grad=True)
a

tensor([ 0.8241,  0.0593, -0.1058], requires_grad=True)

必须指出维度

In [56]:
p = F.softmax(a,dim=0)
p

tensor([0.5376, 0.2502, 0.2122], grad_fn=<SoftmaxBackward0>)

4. 求梯度时，必须指明size，否则会报错：**传入一个相同维度的张量即可**
    - 而且下一行代码如果运行2遍会报错，必须用`retain_graph=True`指明说，计算图要保留，否则算了一次之后就销毁了

In [54]:
p.backward(torch.ones(a.shape))

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

5. 注意这里不能直接传入p去求grad，当对loss求导时，loss一定是一个scalar，不能是一个vector
    - p是vector，p[1]是scalar。当对p[1]求导，返回值是一个dim为1长度为3的，表明这个loss对3个params的更新
    - 当p[1]对3个params求导时，i=j时为正否则为负

In [60]:
p = F.softmax(a,dim=0)
print(p)
print("当p[1]对3个params求导时，i=j时为正否则为负")
torch.autograd.grad(p[1],[a],retain_graph=True)

tensor([0.5376, 0.2502, 0.2122], grad_fn=<SoftmaxBackward0>)
当p[1]对3个params求导时，i=j时为正否则为负


(tensor([-0.1345,  0.1876, -0.0531]),)